In [13]:
import pandas as pd
from psycopg2 import connect
from sqlalchemy import create_engine

# class Configuration:
#     PG_HOST = 'business-intelligent-db.cw1neqwhyrda.eu-central-1.rds.amazonaws.com'
#     PG_USER = 'postgres'
#     PG_PWD = '52eXzbKbqmY2x45f'

PROCESSING_FILE_NAME = '1988.csv'
DB_NAME = 'airline'
TABLE_NAME = 'flight'

class Configuration:
    PG_HOST = 'localhost'
    PG_USER = 'postgres'
    PG_PWD = 'gherukdnbi6lrn4ieljb549ew9j4v98le5j9v48vl5ee'
    
engine = create_engine(f"postgresql://{Configuration.PG_USER}:{Configuration.PG_PWD}@{Configuration.PG_HOST}:5432/{DB_NAME}")
conn = engine.connect()
psy_conn = connect(host=Configuration.PG_HOST,
                   user=Configuration.PG_USER,
                   password=Configuration.PG_PWD,
                   database="airline",
                   port=5432)


In [7]:
import os
import psycopg2

def copy_from_file(conn, table):
    # save the dataframe on disk as a csv file, load
    # the csv file and use copy_from() to copy it to the table
    tmp_df = f"./result/{PROCESSING_FILE_NAME}"
    # df.to_csv(tmp_df, header=True, index=False)
    f = open(tmp_df, 'r')
    cursor = conn.cursor()
    try:
        cursor.copy_from(f, table, sep=',')
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        # os.remove(tmp_df)
        print("Error: %s" %error)
        conn.rollback()
        conn.close()
        return 1
    print("copy_from_file() done")
    cursor.close()
    # os.remove(tmp_df)

In [8]:
# connect to our postgres DB
conn = connect(host=Configuration.PG_HOST,
                user=Configuration.PG_USER,
                password=Configuration.PG_PWD,
                database="airline",
                port=5432)
copy_from_file(conn, 'flight')

Error: invalid input syntax for type integer: "year"
CONTEXT:  COPY flight, line 1, column year: "year"



1

## Load data from csv

In [10]:
processed_flight_df = f"./result/{PROCESSING_FILE_NAME}"
flight_df = pd.read_csv(processed_flight_df)
flight_df.dtypes

<bound method NDFrame.head of          year  month  day_of_month  day_of_week             dep_time  \
0        1988      1             9            6  1988-01-09 13:48:00   
1        1988      1            10            7  1988-01-10 13:34:00   
2        1988      1            11            1  1988-01-11 14:46:00   
3        1988      1            12            2  1988-01-12 13:34:00   
4        1988      1            13            3  1988-01-13 13:41:00   
...       ...    ...           ...          ...                  ...   
5046842  1988     12            12            1  1988-12-12 13:25:00   
5046843  1988     12            13            2  1988-12-13 13:25:00   
5046844  1988     12            14            3  1988-12-14 13:25:00   
5046845  1988     12             1            4  1988-12-01 20:27:00   
5046846  1988     12             2            5  1988-12-02 21:06:00   

                crs_dep_time             arr_time         crs_arr_time  \
0        1988-01-09 13:31:00  1

In [15]:
# Save df to db
flight_df.to_sql(TABLE_NAME, engine, if_exists='replace', index=False)

psy_conn.autocommit = True
cursor = psy_conn.cursor()
  
sql1 = 'SELECT count(*) from flight;'
cursor.execute(sql1)
for i in cursor.fetchall():
    print(i)

# conn.commit()
conn.close()

(5046847,)
